In [52]:
import sys
#sys.path.append('/path/to/project')
sys.path.append("C:/Users/mmahmoud/localGPT")

In [53]:
import ingest
import run_localGPT
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI

import requests
from googlesearch import search
import hashlib
import os
import base64

import pandas as pd
import json
from bs4 import BeautifulSoup

import gradio as gr

In [54]:
CoinMarketCap_DEX_page_URL = "https://coinmarketcap.com/rankings/exchanges/dex/"

In [55]:
def scrape_coinmarketcap_dex_page():
    """
    Scrapes the CoinMarketCap DEX page for the DEX table and writes the data to a xlsx file.
    the table contains the name of the DEX and the link to the DEX page on CoinMarketCap.
    url: URL of the CoinMarketCap DEX page
    save_path: path to save the xlsx file
    """

    # Parse the HTML content
    html_content = requests.get(CoinMarketCap_DEX_page_URL).content
    soup = BeautifulSoup(html_content, "html.parser")

    # Find the table and table rows containing the DEX information
    table = soup.find("table", {"class": "sc-66133f36-3 dOrjhR cmc-table"})
    # table's body
    table_body = table.find("tbody")
    table_rows = table_body.find_all("tr")

    # Prepare the data for the table
    table_data = []
    for row in table_rows:
        columns = row.find_all("td")
        if len(columns) >= 2:
            dex_name_elem = columns[1]
            dex_website_elem = columns[1].find("a", {"class": "cmc-link"})
            if dex_name_elem and dex_website_elem:
                dex_name = dex_name_elem.text
                table_data.append(dex_name)

    # Write the data to a xlsx file
    df = pd.DataFrame(table_data, columns=["Dex Name"])

    # Preporcessing dex names : 
    # - Uniswap v3 (Ethereum)2	-> Uniswap v3
    # - if the dex id from the top 10 remove the last char if it is a digit (classement)

    df["Dex Name"][:10] = df["Dex Name"][:10].apply(lambda x: x[:-1] if x[-1].isdigit() else x)
    df["Dex Name"] = df["Dex Name"].apply(lambda x: x.split("(")[0].strip())

    # delete duplicates based on name and show the percentage of duplicates
    #print("Percentage of duplicates : ", 100 - len(df.drop_duplicates(subset=['Dex Name'], keep='first'))/len(df)*100, "%")
    df = df.drop_duplicates(subset=['Dex Name'], keep='first')

    return df

In [56]:
#df_dex_list = scrape_coinmarketcap_dex_page()

In [57]:
#df_dex_list

In [58]:
#df_dex_list.to_excel("dataframes/dex_list.xlsx", index=False)

In [59]:
# Replace with your GitHub personal access token
TOKEN = '*********************************'

# Define the folder for storing database
SOURCE_DIRECTORY = f"tmp_data"
# Define the folder for storing the embeddings
PERSIST_DIRECTORY = f"tmp_persist"

In [60]:
def save_url_as_html(url, save_path):
    try:
        # Send a GET request to the URL to fetch the content
        response = requests.get(url, timeout=10)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Save the content as an HTML file
            with open(save_path, 'w', encoding='utf-8') as html_file:
                html_file.write(response.text)
            print(f"HTML content saved as {save_path}")
        else:
            print(f"Failed to fetch the URL. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

def get_documents(dex_name, save_path=SOURCE_DIRECTORY):
    # Extract links for liquidity model using googlesearch (only html files)
    query = f'{dex_name} liquidity model'
    search_results = search(query, num_results=1)
    liquidity_model_link = list(search_results)

    # create a folder for the dex liquidity model
    os.makedirs(f'{save_path}/{dex_name}/liquidity_model', exist_ok=True)

    # If no link is found, create a txt file with the message
    if len(liquidity_model_link) == 0:
        with open(f'{save_path}/{dex_name}/liquidity_model/no_liquidity_model.txt', 'w') as f:
            f.write("No liquidity model found for this DEX.")
    
    else:
        # save liquidity model pages as html
        for i, link in enumerate(liquidity_model_link):
            try:
                save_url_as_html(link, f'{save_path}/{dex_name}/liquidity_model/{hashlib.md5(link.encode()).hexdigest()}_{i+1}.html')
            except:
                print("Could not save the page.")

    # create a folder for the dex if it doesn't exist
    os.makedirs(f'{save_path}/{dex_name}/license', exist_ok=True)
    # Flag to track if a license has been found for this DEX
    license_found = False
    
    # Make a GitHub API repository search request based on the DEX name
    search_url = f'https://api.github.com/search/repositories?q={dex_name}&per_page=10'
    headers = {'Authorization': f'token {TOKEN}'}
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        search_results = response.json()['items']

        for repo in search_results:
            # Check if a license file exists and retrieve the license text
            license_url = f'https://api.github.com/repos/{repo["owner"]["login"]}/{repo["name"]}/license'
            response = requests.get(license_url, headers=headers)

            if response.status_code == 200:
                license_data = response.json()
                if 'content' in license_data:
                    license_text = base64.b64decode(license_data['content']).decode('utf-8')
                    # Save license text to a file in the dex folder in the license folder 
                    with open(f'{save_path}/{dex_name}/license/{repo["full_name"].replace("/", "__")}.txt', 'w') as f:
                        f.write(license_text)
                    # Set the flag to True to indicate that a license has been found
                    license_found = True
                    break  # Stop searching for licenses in other repositories for this DEX
            else:
                print(f'Failed to fetch license for {repo["full_name"]}: {response.status_code}')
    
        # If no official license is found, create a txt file with the message
        if not license_found:
            with open(f'{save_path}/{dex_name}/license/no_license.txt', 'w') as f:
                f.write("No official license found for this DEX.")
    else:
        print(f'Failed to search for repositories related to {dex_name}: {response.status_code}')
    
    # return path to the dex folder
    return f'{save_path}/{dex_name}'

In [61]:
MODEL_ID="TheBloke/Mistral-7B-v0.1-GGUF"
MODEL_BASENAME="mistral-7b-v0.1.Q8_0.gguf"
# If llama in model_id, use promptTemplate_type="llama"
promptTemplate_type=None
if "llama" in MODEL_ID.lower():
    promptTemplate_type="llama"
elif "mistral" in MODEL_ID.lower():
    promptTemplate_type="mistral"

EMBEDDING_NAME = "hkunlp/instructor-large"

OPENAI_API_KEY = "*********************************"
OPENAI_ORGANIZATION = "*********************************"

In [62]:
def user_interaction(dex_name, k, co, cs, progress=gr.Progress()):
    results = {}
    # Define features to process
    features = ["liquidity_model", "license"]

    # Check if all parameters are provided
    if dex_name and k is not None and co is not None and cs is not None:
        progress(0.0, desc="Scraping documents...")
        # time.sleep(1)
        # if the dex folder doesn't exist, use get_documents to scrape the documents
        dex_folder = f"{SOURCE_DIRECTORY}/{dex_name}"
        if not os.path.exists(dex_folder):
            dex_folder = get_documents(dex_name)
        # dex_folder = get_documents(dex_name) # SOURCE_DIRECTORY/dex_name
        

        progress(0.1, desc="Loading embedding model...")
        # time.sleep(1)
        # embedding_model = HuggingFaceInstructEmbeddings(model_name=EMBEDDING_NAME, model_kwargs={"device": "cpu"})
        embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, openai_organization=OPENAI_ORGANIZATION)

        progress(0.2, desc="Loading LLM model...")
        #time.sleep(1)
        # llm = run_localGPT.load_model(device_type="cpu", model_id=MODEL_ID, model_basename=MODEL_BASENAME)
        llm = OpenAI(openai_api_key=OPENAI_API_KEY, openai_organization=OPENAI_ORGANIZATION, temperature=0.0)
        
        for i, feature in enumerate(features):
            source_directory = f"{dex_folder}/{feature}"
            progress(0.4 + 0.6*(i)/len(features), desc=f"Processing {feature}..")

            #save_path = f"{source_directory}/{embedding_model.model_name}"
            save_path = f"{source_directory}/openaiembeddings"
            #save_path = f"{PERSIST_DIRECTORY}/{dex_name}/{feature}/{embedding_model.model_name.replace('/', '_')}"
            save_path = f"{PERSIST_DIRECTORY}/{dex_name}/{feature}/openaiembeddings"
            # Convert chunk_size and chunk_overlap to integers
            cs = int(cs)
            co = int(co)
            ingest.main(embedding_model=embedding_model, chunk_size=cs, chunk_overlap=co,
                        source_directory=source_directory, save_path=save_path)
            
            persist_directory = os.path.join(save_path, f'cs_{cs}_co_{co}')

            # Getting the query from queries/feature.txt
            with open(f"../queries/{feature}.txt", "r") as f:
                query = f.read()
                query = query.replace("the DEX", dex_name)

            # Convert k to an integer
            k = int(k)

            # Running localGPT
            answer, docs = run_localGPT.main(llm, embedding_model, k, persist_directory, query, verbose=False, show_sources=False, promptTemplate_type=None)

            # Store the results
            results[feature] = {"answer": answer, "sources": [document.page_content for document in docs]}

            # save the results to a json file
            results_path = f"json/{dex_name}.json"
            os.makedirs("json", exist_ok=True)
            with open(results_path, "w") as f:
                json.dump(results, f)

        # After obtaining the results, update the DataFrame
        
        # Assuming you have obtained features from the results
        features = list(results.keys())
        new_row = {"Dex name": dex_name}

        for feature in features:
            format_answer = results[feature]['answer'].replace('\n', '<br>')
            # enumerate the sources
            sources = [source.replace('\n', ' ') for source in results[feature]['sources']]
            format_sources = "<br>".join([f"<span style='color: Red;'><strong>{i+1}.</strong></span> {source}" for i, source in enumerate(sources)])
            new_row[feature] = f"<span style='color: green;'><strong>Answer:</strong></span> {format_answer}\
                <br> <span style='color: Red;'><strong>Sources:</strong></span><br> {format_sources}"


        # Append the new row to the DataFrame
        df = pd.read_excel("dataframes/table.xlsx")
        # Concatenate the new row with the DataFrame if the row doesn't already exist
        # Otherwise, update the row
        if dex_name in df["Dex name"].values:
            for feature in features:
                df.loc[df["Dex name"] == dex_name, feature] = new_row[feature]
        else:
            df = pd.concat([df, pd.DataFrame(new_row, index=[0])])
        df.to_excel("dataframes/table.xlsx", index=False)

        table = gr.Dataframe(
            headers=["Dex name", "liquidity_model", "license"],
            datatype=["str", "markdown", "markdown"],
            value=pd.read_excel("dataframes/table.xlsx"), 
            wrap=True
            )
        # Unload the model and free up resources
        #del llm
        dex_to_search_from_table = gr.Dropdown(
                                    label="DEX Name",
                                    choices=pd.read_excel("dataframes/table.xlsx")["Dex name"].tolist(),
                                    info="Choose the DEX to search from the table."
                                    )
    return table, results, dex_to_search_from_table

In [36]:
solution1 = "images/chatwdoc (1).png"
solution2 = "images/chatwdoc (2).png"

In [37]:
# Create table.xlsx if it doesn't exist
if not os.path.exists("dataframes/table.xlsx"):
    df = pd.DataFrame({"Dex name": [], "liquidity_model": [], "license": []})
    df.to_excel("dataframes/table.xlsx", index=False)

In [38]:
"""def refresh_table():
    table = gr.Dataframe(
            headers=["Dex name", "liquidity_model", "license"],
            datatype=["str", "markdown", "markdown"],
            value=pd.read_excel("dataframes/table.xlsx"), 
            wrap=True
            )
    return table"""

def refresh_dex_list():
    df_dex_list = pd.read_excel("dataframes/dex_list.xlsx")
    df = scrape_coinmarketcap_dex_page()
    # add def to df_dex_list if not already present
    #df_dex_list = df_dex_list.append(df[~df["Dex Name"].isin(df_dex_list["Dex Name"])])
    # Assuming df and df_dex_list are your DataFrames
    df_dex_list = pd.concat([df_dex_list, df[~df["Dex Name"].isin(df_dex_list["Dex Name"])]])
    # Reset the index of the resulting DataFrame
    df_dex_list.reset_index(drop=True, inplace=True)

    df_dex_list.to_excel("dataframes/dex_list.xlsx", index=False)
    dropdown = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist(),
                value="Uniswap v3",
                allow_custom_value=True,
                filterable=True)
    return dropdown


def delete_table(confirm_checkbox):
    if confirm_checkbox:         
        df = pd.DataFrame({"Dex name": [], "liquidity_model": [], "license": []})
        df.to_excel("dataframes/table.xlsx", index=False)
    else:
        gr.Info("Please confirm that you want to delete the table.")
    table = gr.Dataframe(
            headers=["DEX name", "liquidity_model", "license"],
            datatype=["str", "markdown", "markdown"],
            value=pd.read_excel("dataframes/table.xlsx"), 
            wrap=True
            )
    dropdown = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/table.xlsx")["Dex name"].tolist())
    dex_to_search_from_table = gr.Dropdown(
                                    label="DEX Name",
                                    choices=pd.read_excel("dataframes/table.xlsx")["Dex name"].tolist(),
                                    info="Choose the DEX to search from the table."
                                    )
    return dex_to_search_from_table, dropdown, table

def delete_dex_from_table(dex_to_delete_from_table, confirm_delete_dex_from_table):
    if not dex_to_delete_from_table:
        gr.Info("Please select a DEX to delete from the table.")
    else:
        if not confirm_delete_dex_from_table:
            gr.Info("Please confirm that you want to delete the DEX from the table.")
        else:
            df = pd.read_excel("dataframes/table.xlsx")
            df = df[df["Dex name"] != dex_to_delete_from_table]
            df.to_excel("dataframes/table.xlsx", index=False)
    table = gr.Dataframe(
            headers=["DEX name", "liquidity_model", "license"],
            datatype=["str", "markdown", "markdown"],
            value = pd.read_excel("dataframes/table.xlsx"), 
            wrap=True,
            height=1000
        )
    dropdown = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/table.xlsx")["Dex name"].tolist())
    
    dex_to_search_from_table = gr.Dropdown(
                                    label="DEX Name",
                                    choices=pd.read_excel("dataframes/table.xlsx")["Dex name"].tolist(),
                                    info="Choose the DEX to search from the table."
                                    )
    return dex_to_search_from_table, dropdown, table

def delete_dex_from_dropdown(dex_to_delete_from_dropdown, confirm_delete_dex_from_dropdown):
    if not dex_to_delete_from_dropdown:
        gr.Info("Please select a DEX to delete from the dropdown.")
    else:
        if not confirm_delete_dex_from_dropdown:
            gr.Info("Please confirm that you want to delete the DEX from the dropdown.")
        else:
            df = pd.read_excel("dataframes/dex_list.xlsx")
            df = df[df["Dex Name"] != dex_to_delete_from_dropdown]
            df.to_excel("dataframes/dex_list.xlsx", index=False)
    dropdown = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist(),
                value="Uniswap v3",
                allow_custom_value=True,
                filterable=True)
    dropdown1 = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist()
                )
    return dropdown, dropdown1

def delete_dropdown_list(confirm_delete_dropdown_list):
    if not confirm_delete_dropdown_list:
        gr.Info("Please confirm that you want to delete the dropdown list.")
    else:
        df = pd.DataFrame({"Dex Name": []})
        df.to_excel("dataframes/dex_list.xlsx", index=False)
    dropdown = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist(),
                value="Uniswap v3",
                allow_custom_value=True,
                filterable=True)
    dropdown1 = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist()
                )
    return dropdown, dropdown1

In [39]:
def update_and_extract_all(k, co, cs, progress=gr.Progress()):
    results = gr.JSON(label="Results")
    old_list=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist()
    #dropdown = refresh_dex_list()
    #updated_list=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist()
    #new_list = list(set(updated_list) - set(old_list))
    #if len(new_list) == 0:
        #gr.Info("No new DEXs added to the dropdown.")
    # Show the new DEXs in the dropdown
    #else :
        #gr.Info(f"New DEXs added to the dropdown: {', '.join(new_list)}")
    for dex_name in progress.tqdm(old_list):
        _, results, _ = user_interaction(dex_name, k, co, cs)
    dropdown = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist(),
                value="Uniswap v3",
                allow_custom_value=True,
                filterable=True)
    dropdown1 = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist()
                )
    dropdown2 = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/table.xlsx")["Dex name"].tolist())
    dropdown3 = gr.Dropdown(
                label="DEX Name",
                choices=pd.read_excel("dataframes/table.xlsx")["Dex name"].tolist())
    
    table = gr.Dataframe(
            headers=["DEX name", "liquidity_model", "license"],
            datatype=["str", "markdown", "markdown"],
            value = pd.read_excel("dataframes/table.xlsx"),
            wrap=True,
            height=1000
        )
    return dropdown, dropdown1, dropdown2, dropdown3, results, table

def search_fn(dex_name):
    dex_row = pd.DataFrame({"Dex name": [], "liquidity_model": [], "license": []})
    visible = False
    if not dex_name:
        gr.Info("Please select a DEX to search for.")
    else:
        visible = True
        dex_row = pd.read_excel("dataframes/table.xlsx")[pd.read_excel("dataframes/table.xlsx")["Dex name"] == dex_name]
    return gr.Column(visible=visible), gr.DataFrame(
                    headers=["DEX name", "liquidity_model", "license"],
                    datatype=["str", "markdown", "markdown"],
                    value=dex_row,
                    wrap=True,
                    height=1000
                    )

In [63]:
# Delete dex names into the dropdown list that are in the table
#df_dex_list = pd.read_excel("dataframes/dex_list.xlsx")
#df_table = pd.read_excel("dataframes/table.xlsx")
#df_dex_list = df_dex_list[~df_dex_list["Dex Name"].isin(df_table["Dex name"])]
#df_dex_list.to_excel("dataframes/dex_list.xlsx", index=False)

In [51]:
with gr.Blocks(theme=gr.themes.Default(primary_hue='indigo', secondary_hue='orange')) as demo:
    # title
    gr.Markdown("<h1 style='color: #6C63FF; font-size: 40px; text-align: center;'>DEX explorer</h1>")
    gr.Markdown("<p style='font-size: 20px; text-align: center; margin-top: 10px;'>An app that helps you find answers to questions about a DEX.</p>")

    with gr.Tab("Table"):
        table = gr.Dataframe(
            headers=["DEX name", "liquidity_model", "license"],
            datatype=["str", "markdown", "markdown"],
            value = pd.read_excel("dataframes/table.xlsx"), 
            wrap=True,
            height=1000
        )
        #refresh_button = gr.Button("Refresh Table")
        #refresh_button.click(refresh_table, outputs=[table], show_progress=True)
    with gr.Tab("Search"):
        with gr.Row():
            dex_to_search_from_table = gr.Dropdown(
                                    label="DEX Name",
                                    choices=pd.read_excel("dataframes/table.xlsx")["Dex name"].tolist(),
                                    info="Choose the DEX to search from the table."
                                    )
            search_button = gr.Button("Search", variant='primary')
        with gr.Column(visible=False) as search_column:
            search_result = gr.DataFrame(
                headers=["DEX name", "liquidity_model", "license"],
                datatype=["str", "markdown", "markdown"],
            )
        search_button.click(search_fn, inputs=[dex_to_search_from_table], outputs=[search_column, search_result], show_progress=True)

    with gr.Tab("Interact with the app"):
        with gr.Column():
            with gr.Row():
                with gr.Column():
                    with gr.Row():
                        dex = gr.Dropdown(
                            label="DEX Name", 
                            choices=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist(),
                            value="Uniswap v3",
                            allow_custom_value=True,
                            filterable=True
                            )
                        update_dex_list_button = gr.Button("Update DEX List", variant='primary')
                    gr.Slider(minimum=0, maximum=1, value=0, label="Temperature", info="Choose between 0 and 1")
                    gr.Slider(minimum=0, maximum=1, value=0, label="Top P", info="Choose between 0 and 1")
                with gr.Column("Ingesting Parameters"):
                    cs = gr.Number(label="Chunk Size", value=500) 
                    co = gr.Number(label="Chunk Overlap", value=100)
                    k = gr.Number(label="Number of Chunks", minimum=1, maximum=5, value=3, info="Choose between 1 and 5")            
            with gr.Column():
                results = gr.JSON(label="Results")
        with gr.Row():
            extract_button = gr.Button("Extract", variant='primary')
            update_and_extract_all_button = gr.Button("Update and Extract All", variant='primary')

        extract_button.click(user_interaction, inputs=[dex, k, co, cs], outputs=[table, results, dex_to_search_from_table])
        update_dex_list_button.click(refresh_dex_list, outputs=[dex], show_progress=True)

    # Add a new advanced features tab
    # Theses features are : deleting a dex name from the dropdown list, deleting a dex_name from the table
    # deleting the entire table, deleting the entire dropdown list
    with gr.Tab("Delete"):
        with gr.Row():
            dex_to_delete_from_dropdown = gr.Dropdown(
                                        label="DEX Name",
                                        choices=pd.read_excel("dataframes/dex_list.xlsx")["Dex Name"].tolist(),
                                        info="Choose the DEX to delete from the dropdown list."
                                        )
            with gr.Column():
                confirm_delete_dex_from_dropdown = gr.Checkbox(
                                                    label="Confirm",
                                                    info="This will delete the DEX from the dropdown \
                                                        list and cannot be undone. By checking this box, \
                                                        you confirm that you want to delete the DEX from the dropdown list."
                                                    )
                delete_dex_from_dropdown_button = gr.Button("Delete DEX from dropdown list", variant='primary')
        with gr.Row():
            dex_to_delete_from_table = gr.Dropdown(
                                    label="DEX Name",
                                    choices=pd.read_excel("dataframes/table.xlsx")["Dex name"].tolist(),
                                    info="Choose the DEX to delete from the table."
                                    )
            with gr.Column():
                confirm_delete_dex_from_table = gr.Checkbox(
                                                label="Confirm",
                                                info="This will delete the DEX from the table and \
                                                    cannot be undone. By checking this box, you \
                                                    confirm that you want to delete the DEX from the table."
                                                    )
                delete_dex_from_table_button = gr.Button("Delete DEX from table", variant='primary')
        with gr.Row():
            with gr.Column():
                confirm_delete_table = gr.Checkbox(label="Confirm", info="This will delete the table and cannot be undone. By checking this box, you confirm that you want to delete the table.")
                delete_table_button = gr.Button("Delete Table", variant='primary')
            with gr.Column():
                confirm_delete_dropdown_list = gr.Checkbox(label="Confirm", info="This will delete the dropdown list and cannot be undone. By checking this box, you confirm that you want to delete the dropdown list.")
                delete_dropdown_list_button = gr.Button("Delete Dropdown List", variant='primary')
        
        delete_dex_from_table_button.click(delete_dex_from_table, inputs=[dex_to_delete_from_table, confirm_delete_dex_from_table], outputs=[dex_to_search_from_table, dex_to_delete_from_table, table], show_progress=True)
        delete_dex_from_dropdown_button.click(delete_dex_from_dropdown, inputs=[dex_to_delete_from_dropdown, confirm_delete_dex_from_dropdown], outputs=[dex, dex_to_delete_from_dropdown], show_progress=True)
        delete_dropdown_list_button.click(delete_dropdown_list, inputs=[confirm_delete_dropdown_list], outputs=[dex, dex_to_delete_from_dropdown], show_progress=True)
        delete_table_button.click(delete_table, inputs=[confirm_delete_table], outputs=[dex_to_search_from_table, dex_to_delete_from_table, table], show_progress=True)

    update_and_extract_all_button.click(
            update_and_extract_all,
            inputs=[k, co, cs],
            outputs=[dex, dex_to_delete_from_dropdown, dex_to_delete_from_table, dex_to_search_from_table, results, table],
            show_progress=True
            )

    with gr.Tab("How does it work?"):
        gr.Gallery(label="Solution", value=[solution1, solution2], columns=2, rows=1, object_fit="scale-down")

demo.queue(concurrency_count=20).launch()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Loading documents from tmp_data/Noah/liquidity_model
Loaded 2 documents from tmp_data/Noah/liquidity_model
Split into 157 chunks of text
Loading documents from tmp_data/Noah/license
Loaded 1 documents from tmp_data/Noah/license
Split into 35 chunks of text
Loading documents from tmp_data/Sharkyswap/liquidity_model
Loaded 2 documents from tmp_data/Sharkyswap/liquidity_model
Split into 41 chunks of text
Loading documents from tmp_data/Sharkyswap/license
Loaded 1 documents from tmp_data/Sharkyswap/license
Split into 103 chunks of text
Loading documents from tmp_data/Symmetric/liquidity_model
Loaded 2 documents from tmp_data/Symmetric/liquidity_model
Split into 185 chunks of text
Loading documents from tmp_data/Symmetric/license
Loaded 1 documents from tmp_data/Symmetric/license
Split into 3 chunks of text
Loading documents from tmp_data/Kwikswap/liquidity_model
Loaded 3 documents from tmp_data/Kwikswap/liquidity_model
Split into 123 chunks of text
Loading documents from tmp_data/Kwikswap/

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Loading documents from tmp_data/CantoSwap/liquidity_model
Loaded 3 documents from tmp_data/CantoSwap/liquidity_model
Split into 104 chunks of text
Loading documents from tmp_data/CantoSwap/license
Loaded 1 documents from tmp_data/CantoSwap/license
Split into 1 chunks of text


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


...

Loading documents from tmp_data/Apeswap/license
Loaded 2 documents from tmp_data/Apeswap/license
Split into 104 chunks of text
Loading documents from tmp_data/ALEX/liquidity_model
Loaded 3 documents from tmp_data/ALEX/liquidity_model
Split into 151 chunks of text
Loading documents from tmp_data/ALEX/license
Loaded 1 documents from tmp_data/ALEX/license
Split into 3 chunks of text
Loading documents from tmp_data/Oasis Swap/liquidity_model
Loaded 3 documents from tmp_data/Oasis Swap/liquidity_model
Split into 61 chunks of text
Loading documents from tmp_data/Oasis Swap/license
Loaded 1 documents from tmp_data/Oasis Swap/license
Split into 1 chunks of text


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Loading documents from tmp_data/NESTFi/liquidity_model
Loaded 3 documents from tmp_data/NESTFi/liquidity_model
Split into 33 chunks of text
Loading documents from tmp_data/NESTFi/license
Loaded 1 documents from tmp_data/NESTFi/license
Split into 3 chunks of text
Loading documents from tmp_data/Merlin DEX/liquidity_model
Loaded 2 documents from tmp_data/Merlin DEX/liquidity_model
Split into 53 chunks of text
Loading documents from tmp_data/Merlin DEX/license
Loaded 1 documents from tmp_data/Merlin DEX/license
Split into 1 chunks of text


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Loading documents from tmp_data/ko.one/liquidity_model
Loaded 3 documents from tmp_data/ko.one/liquidity_model
Split into 41 chunks of text
Loading documents from tmp_data/ko.one/license
Loaded 1 documents from tmp_data/ko.one/license
Split into 31 chunks of text
Loading documents from tmp_data/Purple Bridge/liquidity_model
Loaded 3 documents from tmp_data/Purple Bridge/liquidity_model
Split into 153 chunks of text
Loading documents from tmp_data/Purple Bridge/license
Loaded 1 documents from tmp_data/Purple Bridge/license
Split into 103 chunks of text
